## Ejemplo de Quadtree

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oscar-ramos/robotica-autonoma-python/blob/main/6-Planificacion-Movimiento/6-04-Quadtree.ipynb)

In [1]:
import random
import numpy as np
from pqdict import pqdict

from PIL import Image, ImageFilter, ImageTk, ImageDraw
import tkinter as tk

### Función para la creación de un mapa aleatorio

In [2]:
def generate_map(size, kernelsize, numiterations, col_impassable, col_passable):
    """
    Generate random map
    """
    im = Image.new('RGB', (size, size), color=col_impassable)
    # Initialize with random data
    for x in range(0, im.width):
        for y in range(0, im.height):
            im.putpixel((x, y), random.choice([col_impassable, col_passable]))
    # Apply filter multiple times
    for i in range(numiterations):
        im = im.filter(ImageFilter.RankFilter(kernelsize, kernelsize**2 // 2))
    return im

### Implementación de un Quadtree

In [3]:
# Flags
UL = 0; UR = 1; LL = 2; LR = 3

class Quadtree(object):
    """
    QuadTree on 2D image map
    
    Recursive data structure. Class serves as the root quadtree, subtrees and leaf Tiles.
    A Tile has either exactly four child Tiles or no child Tiles.
    Tiles with no child Tiles are leaf Tiles and have a uniform color.
    
    """
    def __init__(self, image, pos=UL, level=0, x=0, y=0, limit=None, col=None):
        self.level = level
        self.color_passable = col
        self.bb = BoundingBox(x, y, image.width, image.height)
        self.childs = None      # Either a tile has four child tiles
        self.color = None       # or a final color value      
        self._setimage(image, limit)

    def center(self):
        return self.bb.center()

    def depth(self):
        """
        Depth of quadtree/subtree
        """
        if not self.childs:
            return self.level
        else:
            return max((child.depth() for child in self.childs))
       
    def count(self, cnt = 0):
        """
        Number of tiles in quadtree/subtree
        """
        if not self.childs:
            return 1
        else:
            return 1 + sum((child.count() for child in self.childs))

    def get(self, x, y):
        """
        Get leaf Tile containing position x,y from quadtree/subtree
        """
        if not self.childs:
            return self
        else:
            s = self.bb.w // 2
            if x < self.bb.x + s:
                if y < self.bb.y + s:
                    return self.childs[UL].get(x, y)
                else:
                    return self.childs[LL].get(x, y)
            else:
                if y < self.bb.y + s:
                    return self.childs[UR].get(x, y)
                else:
                    return self.childs[LR].get(x, y)

    def intersect(self, bbox, retlist):
        """
        Intersect quadtree/subtree with BoundingBox
        return list of intersecting i.e. overlapping leaf Tiles
        """ 
        if not self.bb.intersects(bbox):
            return retlist
        
        if not self.childs:
            return retlist.append(self)
        else:
            self.childs[UL].intersect(bbox, retlist)
            self.childs[UR].intersect(bbox, retlist)
            self.childs[LL].intersect(bbox, retlist)
            self.childs[LR].intersect(bbox, retlist)
        
    def _setimage(self, image, limit):
        if len(image.getcolors()) == 1:
            # The image is uniformly colored. Tile is a leaf (without children)
            # Recursion ends here. Color is the uniform color of the image
            self.color = image.getcolors()[0][1]
        else:            
            # Image is not uniformly colored (it has more than one color)
            if limit and self.level >= limit:
                # Depth limit is given and Tile is at limit. Forces a leaf Tile
                # End recursion here and (arbitrary) set color of Tile
                self.color = self.color_passable
            else:
                # Not a leaf Tile: split Tile and do recursion on children
                self._split(image, limit)
    
    def _split(self, image, limit):
        # Split Tile into four child tiles
        self.childs = [None, None, None, None]
        # Split image at half height and width to create four subimages
        # Create four quadtrees as childs
        s = image.height // 2
        self.childs[UL] = Quadtree(image.crop((0, 0, 0+s, 0+s)), UL, self.level+1, self.bb.x + 0, self.bb.y + 0, limit, 
                                  self.color_passable)
        self.childs[UR] = Quadtree(image.crop((s, 0, s+s, 0+s)), UR, self.level+1, self.bb.x + s, self.bb.y + 0, limit,
                                  self.color_passable)
        self.childs[LL] = Quadtree(image.crop((0, s, 0+s, s+s)), LL, self.level+1, self.bb.x + 0, self.bb.y + s, limit,
                                  self.color_passable)
        self.childs[LR] = Quadtree(image.crop((s, s, s+s, s+s)), LR, self.level+1, self.bb.x + s, self.bb.y + s, limit,
                                  self.color_passable)

    def __repr__(self):
        return "l:{} {},{} {}:{}".format(self.level, self.bb.x, self.bb.y, self.bb.w, self.bb.h)


### Funciones y clases auxiliares del Quadtree

In [4]:
def draw_quadtree(draw, tile, maxdepth):
    if tile.level == maxdepth:
        draw_tile(draw, tile, color=(255, 110, 110))
        return
    
    if tile.childs:
        for child in tile.childs:
            draw_quadtree(draw, child, maxdepth)
    else:
        draw_tile(draw, tile, color=(255, 110, 110))

    
def draw_tile(draw, tile, color):
    draw.rectangle([tile.bb.x, tile.bb.y, tile.bb.x+tile.bb.w, tile.bb.y+tile.bb.h], outline=color)


def fill_tile(draw, tile, color):
    draw.rectangle([tile.bb.x+1, tile.bb.y+1, tile.bb.x+tile.bb.w-1, tile.bb.y+tile.bb.h-1], outline=None, fill=color)
    
    
class BoundingBox:
    """
    Simple bounding box implementation
    """
    def __init__(self, x, y, w, h):
        self.x = x; self.y = y; self.w = w; self.h = h
            
    def intersects(self, o):
        return (self.x < o.x + o.w and self.y < o.y + o.h and
               self.x + self.w > o.x and self.y + self.h > o.y)
    
    def contains(self, x, y):
        return (self.x < x < self.x + self.w and self.y < y < self.y + self.h)

    def center(self):
        return self.x + self.w//2, self.y + self.h//2

### Implementación de A*

In [5]:
def astar(adjafunc, distfunc, heurfunc, start, goal, col_impassable):
    """ 
    adjafunc: node -> List(nodes)
    distfunc: node, node -> double
    start: start node
    goal: end node. Terminate when reached
    color: color of the impassable zone
    
    Return dict node -> absolute dist from start, dict node -> path predessor node
    
    """
    D = {start: 0}           # Final absolute distances
    P = {}                   # Predecessors
    Q = pqdict({start: 0})   # Fringe/frontier maps unexpanded node to estimated dist to goal
    considered = 0           # Count how many nodes have been considered on multiple paths

    # Keep expanding nodes from the fringe until the goal node is reached
    # or no more new nodes can be reached and fringe runs empty
    for n, estimation in Q.popitems():   # Pop node with min estimated costs from queue
        if n == goal:                    # Reached goal node
            break                        # Stop expanding nodes
        for neighb in adjafunc(n, col_impassable):   # For all neighbours/adjacent of current node n
            considered += 1
            # Calculate distance to neighbour: cost to current + cost reaching neighbour from current
            dist = D[n] + distfunc(n, neighb)        
            if neighb not in D or D[neighb] > dist:         # If neighbour never visited or shorter using this way
                D[neighb] = dist                            # Find (shorter) distance to neighbour
                Q[neighb] = dist + heurfunc(neighb, goal)   # Estimate distance from neighbour to goal
                P[neighb] = n                               # Remember we reached neighbour via n  

    # Expanding done: distance map D populated
    if goal not in D:                # goal node not in distance map
        return None, D, considered   # no path to goal found

    # Build path from start to goal by walking backwards on the predecessor map
    path = []              # start with empty path
    n = goal               # at the goal node
    while n != start:      # while not yet at the start node
        path.insert(0, n)  #     prepend node to path
        n = P[n]           #     get predecessor of node
        
    path.insert(0, start)  # dont forget the start node
    return path, D, considered

### Ventana principal (usando Tkinter)

In [6]:
class MainObject(object):
    
    def __init__(self, mapsize, col_impassable, col_passable):
        self.mapsize = mapsize  
        self.color_impassable = col_impassable
        self.color_passable = col_passable
    
    def run(self):
        self.mapimage = None
        self.quadtree = None
        self.startpoint = None
        self.drag_startp = False
        self._setupgui()
        self.root.mainloop()    

    def _setupgui(self):
        self.root = tk.Tk()
        self.root.title("QuadTree A*")
        
        # Canvas for the figure
        # ------------------------
        self.canvas = tk.Canvas(self.root, bg='gray', width=self.mapsize, height=self.mapsize)
        self.canvas.pack(side=tk.LEFT)
        self.image_item = self.canvas.create_image((0, 0), anchor=tk.NW)

        # Frames
        rightframe = tk.Frame(self.root)
        rightframe.pack(side=tk.LEFT, fill=tk.Y)

        # Map Frame (top right)
        # ---------------------
        mapframe = tk.Frame(rightframe, relief=tk.SUNKEN, borderwidth=2)
        mapframe.pack(padx=5, pady=5)
        label = tk.Label(mapframe, text="Map", font=("Helvetica", 13))
        label.pack()

        frame1 = tk.Frame(mapframe)
        frame1.pack(fill=tk.X, padx=4)
        kernellbl = tk.Label(frame1, text="Kernel Size")
        kernellbl.pack(side=tk.LEFT, pady=4)

        self.kernelsizevar = tk.StringVar(self.root)
        self.kernelsizevar.set("7*7")
        kernelmenu = tk.OptionMenu(frame1, self.kernelsizevar, "13*13", "11*11", "9*9", "7*7", "5*5", "3*3")
        kernelmenu.pack(fill=tk.X, expand=True)

        frame2 = tk.Frame(mapframe)
        frame2.pack(fill=tk.X, padx=4)
        
        iterslbl = tk.Label(frame2, text="Num Iterations")
        iterslbl.pack(side=tk.LEFT, pady=4)

        var = tk.StringVar(self.root)
        var.set("40")
        self.iterspin = tk.Spinbox(frame2, from_=0, to=100, textvariable=var)
        self.iterspin.pack(expand=True)

        genbtn = tk.Button(mapframe, text="Generate Map", command=self.onButtonGeneratePress)
        genbtn.pack(pady=2)

        # Quadtree Frame (right)
        # ---------------------
        qtframe = tk.Frame(rightframe, relief=tk.SUNKEN, borderwidth=2)
        qtframe.pack(fill=tk.X, padx=5, pady=5)
        
        label = tk.Label(qtframe, text="QuadTree", font=("Helvetica", 13))
        label.pack()

        frame1 = tk.Frame(qtframe)
        frame1.pack(fill=tk.X, padx=4)

        label = tk.Label(frame1, text="Depth Limit")
        label.pack(side=tk.LEFT, pady=4)

        var = tk.StringVar(self.root)
        var.set("100")        
        self.limitspin = tk.Spinbox(frame1, from_=2, to=100, textvariable=var)
        self.limitspin.pack(expand=True)
        
        self.qtlabelvar = tk.StringVar()
        label = tk.Label(qtframe, fg='#FF8080', textvariable=self.qtlabelvar)
        label.pack()

        quadtreebtn = tk.Button(qtframe, text="Generate QuadTree", command=self.onButtonQuadTreePress)
        quadtreebtn.pack(pady=2)
        
        # Path Frame (right)
        # ---------------------
        astarframe = tk.Frame(rightframe, relief=tk.SUNKEN, borderwidth=2)
        astarframe.pack(fill=tk.X, padx=5, pady=5)
                
        label = tk.Label(astarframe, text="Path", font=("Helvetica", 13))
        label.pack()

        self.pathlabelvar =tk.StringVar()
        label = tk.Label(astarframe, fg='#0000FF', textvariable=self.pathlabelvar)
        label.pack()

        self.astarlabelvar = tk.StringVar()
        label = tk.Label(astarframe, fg='#8080FF', textvariable=self.astarlabelvar)
        label.pack()

        # Instructions Frame (bottom right)
        # ---------------------------------
        label = tk.Label(rightframe, text="Instructions", font=("Helvetica", 13))
        label.pack()
        label = tk.Label(rightframe, justify=tk.LEFT, text=
        "Generate a random map.\n"
        "Black regions are impassable.\n"
        "Generate QuadTree on map.\n"
        "Set start position by dragging blue circle.\n"
        "Click anywhere on map to find a path.")
        label.pack(padx=14)

        self.canvas.bind('<ButtonPress-1>', self.onMouseButton1Press)
        self.canvas.bind('<ButtonRelease-1>', self.onMouseButton1Release)
        self.canvas.bind('<B1-Motion>', self.onMouseMove)

    def onMouseButton1Press(self, event):
        if not self.quadtree:
            return
        if self.startpoint in self.canvas.find_overlapping(event.x, event.y, event.x, event.y):
            self.drag_startp = True
            return
        startx, starty, _, _ = self.canvas.coords(self.startpoint)
        start = self.quadtree.get(startx + 6, starty + 6)
        goal = self.quadtree.get(event.x, event.y)

        adjacent = make_adjacent_function(self.quadtree, self.color_impassable)
        path, distances, considered = astar(adjacent, euclidean_dist, euclidean_dist, 
                                            start, goal, self.color_impassable)

        im = self.qtmapimage.copy()
        draw = ImageDraw.Draw(im)

        self.astarlabelvar.set("Nodes visited: {} considered: {}".format(len(distances), considered))
        for tile in distances:
            fill_tile(draw, tile, color=(0xC0, 0xC0, 0xFF))

        if path:
            self.pathlabelvar.set("Path Cost: {}  Nodes: {}".format(round(distances[goal], 1), len(path)))
            for tile in path:
                fill_tile(draw, tile, color=(0, 0, 255))
        else:
            self.pathlabelvar.set("No Path found.")
        self._updateimage(im)
    
    def onMouseButton1Release(self, event):
        self.drag_startp = False
    
    def onMouseMove(self, event):
        if self.drag_startp:
            self.canvas.coords(self.startpoint, event.x-6, event.y-6, event.x+6, event.y+6)

    def onButtonGeneratePress(self):
        """
        Event for button 'Generate Map'
        """
        ksize = int(self.kernelsizevar.get().split('*')[0])
        numiter = int(self.iterspin.get())
        self.root.config(cursor="watch")
        self.root.update()
        self.mapimage = generate_map(self.mapsize, ksize, numiter,
                                     self.color_impassable, self.color_passable)
        self._updateimage(self.mapimage)
        self.quadtree = None
        self.qtlabelvar.set("") 
        self.canvas.delete(self.startpoint)
        self.startpoint = None
        self.astarlabelvar.set("")
        self.pathlabelvar.set("")
        self.root.config(cursor="")        

    def onButtonQuadTreePress(self):
        """
        Event for button 'Generate QuadTree'
        """
        if not self.mapimage:
            return     
        depthlimit = int(self.limitspin.get())
        # Crear un quadtree
        self.quadtree = Quadtree(self.mapimage, limit=depthlimit, col=self.color_passable)
        self.qtmapimage = self.mapimage.copy()
        draw = ImageDraw.Draw(self.qtmapimage)
        draw_quadtree(draw, self.quadtree, 8)
        self._updateimage(self.qtmapimage)
        
        self.qtlabelvar.set("Depth: {}  Nodes: {}".format(self.quadtree.depth(), self.quadtree.count()))
        self.astarlabelvar.set("")
        self.pathlabelvar.set("")
        
        if not self.startpoint:
            pos = self.mapsize//2
            self.startpoint = self.canvas.create_oval(pos-6, pos-6, pos+6, pos+6, fill='#2028FF', width=2)

    def _updateimage(self, image):
        self.imagetk = ImageTk.PhotoImage(image)
        self.canvas.itemconfig(self.image_item, image=self.imagetk)

### Funciones auxiliares

In [7]:
def euclidean_dist(start, end):
    dx, dy = start.center()[0] - end.center()[0], start.center()[1] - end.center()[1]
    return np.sqrt(dx**2 + dy**2)

def neighbours(qt, tile):
    """
    Return neighbour tiles for tile in quadtree. Intersect the whole quadtree with
    a slightly expanded bounding box of the query tile.
    (There are more efficient ways to find neighbouring tiles in a quadtree)
    
    """
    neigh = []
    qt.intersect(BoundingBox(tile.bb.x - 1, tile.bb.y -1, tile.bb.w + 2, tile.bb.h + 2), neigh)
    return neigh

def make_adjacent_function(quadtree, col_impassable):
    """
    Return a function suitable as adjacent function as parameter to A*
    This wrapper function captures the quadtree in a closure of the adjacent function
    """
    def adjacent(node, col_impassable):
        """
        Return nodes (Tiles) adjacent to node
        Adjacent nodes are not impassable and can be directly reached from node
        """
        a = []
        for neighbour in neighbours(quadtree, node):
            assert neighbour.childs is None             # Must be leaf node
            if neighbour != node and neighbour.color != col_impassable:
                a.append(neighbour)
        return a

    return adjacent

### Programa principal

In [8]:
# Alto y ancho del mapa cuadrado (debe ser una potencia de 2: 256, 512, 1024, etc.)
mapsize = 512

# colores para el mapa
col_impassable = 0, 0, 0         # Para regiones libres (pasable)
col_passable = 220, 220, 220     # Para regiones con obstáculo (no pasable)

# Objeto principal
o = MainObject(mapsize, col_impassable, col_passable)
o.run()